In [64]:
import os
import pandas as pd
import json

folder_path = "/content/drive/MyDrive/Dane z aplikacji/Endomondo/Workouts"

dfs = []

files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

if files:
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                if isinstance(data, list) and len(data) > 0:
                    sport = 'Running' if data[0].get('sport') == 'RUNNING' else data[0].get('sport')
                    if sport == 'Running':
                        source = data[1].get('source')
                        source = source.replace("INPUT_MANUAL", "Endomondo (Manual)").replace("TRACK_MOBILE", "Endomondo (GPS)")

                        start_time = pd.to_datetime(data[3].get('start_time'))
                        end_time = pd.to_datetime(data[4].get('end_time'))
                        speed_avg_kmh = round(float(data[8].get('speed_avg_kmh')), 2) if data[8].get('speed_avg_kmh') is not None else None
                        duration = str(data[5].get('duration_s')).split('.')[0] if data[5].get('duration_s') is not None else None
                        distance = round(float(data[6].get('distance_km')), 2) if duration is not None else None

                        dfs.append({
                            'sport': sport,
                            'source': source,
                            'date': start_time.date(),
                            'start_time': start_time.time(),
                            'end_time': end_time.time(),
                            'duration_s': duration,
                            'distance_km': distance,
                            'speed_avg_kmh': speed_avg_kmh
                        })

            except Exception as e:
                print(f"Error processing JSON file {file_name}: {e}")

    df1 = pd.DataFrame(dfs).sort_values(by='date').reset_index(drop=True)
    display(df1)

else:
    print("No JSON files found in the specified folder.")


,sport,source,date,start_time,end_time,duration_s,distance_km,speed_avg_kmh
0,Running,Endomondo (Manual),2012-08-15,10:07:00,10:33:00,1560,4.21,9.72
1,Running,Endomondo (Manual),2012-08-18,18:07:00,19:04:00,3420,8.48,8.93
2,Running,Endomondo (Manual),2012-08-20,18:30:00,19:19:00,2940,6.73,8.24
3,Running,Endomondo (Manual),2012-08-22,18:30:00,19:24:00,3240,7.77,8.63
4,Running,Endomondo (Manual),2012-08-26,18:21:00,19:13:00,3120,7.77,8.97
...,...,...,...,...,...,...,...,...
646,Running,Endomondo (GPS),2020-10-25,05:47:17,06:46:08,3522,10.07,NaN
647,Running,Endomondo (GPS),2020-10-29,14:59:15,16:00:49,3673,10.03,NaN
648,Running,Endomondo (GPS),2020-11-01,16:52:07,16:52:10,3239,9.44,NaN
649,Running,Endomondo (GPS),2020-11-04,19:53:52,19:53:55,3370,9.37,NaN


In [65]:
import pandas as pd
from datetime import datetime
from dateutil import parser

file_path = "/content/drive/MyDrive/Dane z aplikacji/MapMyRun/user184238914_workout_history.csv"

df = pd.read_csv(file_path, header=None, skip_blank_lines=False)

df2 = df[df[2] == 'Run'].copy()

df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]

display(df2)

,sport,source,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


In [66]:
df2['date'] = pd.to_datetime(df2['date'])

# Utwórz kolumnę z rokiem i miesiącem
df2['year_month'] = df2['date'].dt.to_period('M')

# Użyj funkcji groupby i sum, aby uzyskać łączną liczbę distance_km dla każdego roku-miesiąca tylko dla df2
df3 = df2.groupby('year_month')['distance_km'].sum().reset_index()

display(df3)

,year_month,distance_km
0,2012-08,52.35
1,2012-09,98.59
2,2012-10,91.96
3,2012-11,95.55
4,2012-12,52.71
...,...,...
94,2020-08,131.90
95,2020-09,138.45
96,2020-10,50.11
97,2020-11,27.97


In [88]:
import pandas as pd
import numpy as np

google_sheets_link = 'https://docs.google.com/spreadsheets/d/1sKQRBWY5xvY-erhFxXo0bC0pTpZT6QUUMx1fjLngGk0/edit?usp=drive_link'

document_key = google_sheets_link.split('/')[-2]

worksheet_link = f'https://docs.google.com/spreadsheets/d/{document_key}/export?format=csv'

df4 = pd.read_csv(worksheet_link, header=1)
df4 = df4.drop(columns=['Unnamed: 0'])
df4 = df4.iloc[:, :-1]
df4 = df4.iloc[:-1,:]
df4.replace('-', np.nan, inplace=True)
display(df4)


,miesiąc / rok,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,styczeń,NaN,"76,59","43,08","48,17","65,66","51,16","106,34","0,00","40,75","32,10","47,98","0,00"
1,luty,NaN,"57,04","65,36","83,30","44,66","46,28","17,88","7,79","67,29","15,09","47,70","8,14"
2,marzec,NaN,"41,12","44,55","90,17","138,11","38,07","43,73","48,15","94,18","0,00","18,01","12,17"
3,kwiecień,NaN,"58,38","52,41","64,98","109,17","68,11","73,62","45,31","111,77","72,62","29,20","0,00"
4,maj,NaN,"83,13","89,69","156,73","149,46","39,31","45,77","71,53","108,07","104,15","29,55","26,97"
5,czerwiec,NaN,"77,39","116,45","83,30","94,79","36,29","32,15","36,45","74,35","79,11","10,06","33,88"
6,lipiec,NaN,"73,47","104,91","70,30","93,11","51,82","66,03","77,27","124,15","111,90","0,00","49,16"
7,sierpień,"52,35","51,63","172,00","76,41","66,36","89,59","52,94","47,96","131,90","126,96","5,02","23,22"
8,wrzesień,"98,59","87,30","106,57","104,72","39,38","30,54","60,32","58,86","138,45","120,55","5,06","13,48"
9,październik,"91,96","62,25","73,01","113,72","28,00","69,00","37,71","64,69","50,10","83,80","0,00",NaN


In [126]:
df5 = pd.DataFrame(columns=['rok-miesiac', 'wartosc'])

# Iteruj po wierszach i kolumnach
for index, row in df.iterrows():
    for col in df4.columns[1:]:
        df5 = pd.concat([df5, pd.DataFrame({'rok-miesiac': [f'{int(col)}-{index + 1:02d}'], 'wartosc': [row[col]]})], ignore_index=True)

# Wyświetl wynikowy DataFrame
df5 = df5.rename(columns={'rok-miesiac': 'year_month', 'wartosc': 'distance_km'})

# Sortuj DataFrame według kolumny "year_month"
df5= df5.sort_values(by='year_month')
df5 = df5[df5['year_month'].apply(lambda x: int(x.split('-')[1])) != 13]
display(df5)

,year_month,distance_km
0,2012-01,-
12,2012-02,-
24,2012-03,-
36,2012-04,-
48,2012-05,-
...,...,...
95,2023-08,"23,22"
107,2023-09,"13,48"
119,2023-10,NaN
131,2023-11,NaN


In [135]:
df3['year_month'] = pd.to_datetime(df3['year_month'])
df5['year_month'] = pd.to_datetime(df5['year_month'])

merged_df = pd.merge(df3, df5, on='year_month', suffixes=('_df3', '_df5'))
merged_df['distance_km_df3'] = pd.to_numeric(merged_df['distance_km_df3'])
merged_df['distance_km_df5'] = pd.to_numeric(merged_df['distance_km_df5'].str.replace(',', '.'))
merged_df['absolute_difference'] = abs(merged_df['distance_km_df3'] - merged_df['distance_km_df5'])

filtered_df = merged_df[merged_df['absolute_difference'] >= 0.02].copy()

# Convert 'year_month' column to datetime and then to period
filtered_df['year_month'] = pd.to_datetime(filtered_df['year_month']).dt.to_period('M')

# Display the resulting DataFrame
print(filtered_df)


   year_month  distance_km_df3  distance_km_df5  absolute_difference
58    2017-06            36.27            36.29                 0.02
69    2018-05            45.75            45.77                 0.02
97    2020-11            27.97            74.58                46.61
98    2020-12             1.61            63.38                61.77
